In [1]:
from Preprocessing import Preprocessing
ps = Preprocessing()

images = ['img/marian_0.jpg', 'img/marian_1.jpg']
results = ps.do_it(images)
print('Puzzles: {}'.format(len(results)))

Files: 2
Puzzles: 54


In [2]:
from Solver import Solver
sol = Solver()
sol.solve(results)

NotImplementedError: 